# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [34]:
pip install cartopy geoviews pyproj


     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ------------------------------------ --- 51.2/56.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 56.0/56.0 kB 725.9 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.0 MB 2.9 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.0 MB 3.9 MB/s eta 0:00:03
   - -------------------------------------- 0.5/11.0 MB 5.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/11.0 MB 7.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.7/11.0 MB 8.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.0 MB 8.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/11.0 MB 8.8 MB/s eta 0:00:02
   --------- ----

In [38]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import importlib


# Import API key
from api_keys import geoapify_key


In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longyearbyen,78.2186,15.6401,2.91,87,75,2.57,SJ,1731385859
1,1,atafu village,-8.5421,-172.5159,28.74,72,21,5.94,TK,1731385894
2,2,port-aux-francais,-49.3500,70.2167,5.18,93,100,9.63,TF,1731385894
3,3,puerto ayora,-0.7393,-90.3518,21.42,87,45,1.79,EC,1731385894
4,4,parauapebas,-6.0675,-49.9022,21.96,99,98,0.73,BR,1731385894
...,...,...,...,...,...,...,...,...,...,...
560,560,mansa,29.9833,75.3833,28.90,17,0,3.06,IN,1731385995
561,561,bodo,67.2800,14.4050,7.02,90,75,8.75,NO,1731385995
562,562,beloyarskiy,63.7119,66.6722,-13.34,98,88,1.67,RU,1731385995
563,563,palmerston north,-40.3500,175.6167,17.65,1,27,3.91,NZ,1731385814


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [42]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
data = {
    'City_ID': city_data_df["City_ID"],
    'City': city_data_df["City"],
    'Lat': city_data_df["Lat"],
    'Lng': city_data_df["Lng"],
    'Humidity': city_data_df["Humidity"]
}

city_data_df = pd.DataFrame(data)


scatter_plot = city_data_df.hvplot.scatter(
    x='Lng',  
    y='Lat',  
    size='Humidity',  
    color='Humidity',  
    cmap='Blues',  
    alpha=0.6, 
    hover_cols=['City', 'Humidity'],  
    title="Cities with Humidity vs Latitude",
    xlabel='Longitude',
    ylabel='Latitude'
)

scatter_plot
# Display the map
# YOUR CODE HERE

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
city_data_df.columns

Index(['City_ID', 'City', 'Lat', 'Lng', 'Humidity'], dtype='object')

In [48]:
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= 20) &  
    (city_data_df['Max Temp'] <= 30) &  
    (city_data_df['Humidity'] < 80) &   
    (city_data_df['Cloudiness'] < 30) & 
    (city_data_df['Wind Speed'] <= 10)  
]

ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,atafu village,-8.5421,-172.5159,28.74,72,21,5.94,TK,1731385894
9,9,chaohu,31.6000,117.8667,22.09,40,0,5.50,CN,1731385895
13,13,geraldton,-28.7667,114.6000,24.23,50,0,9.77,AU,1731385896
14,14,new norfolk,-42.7826,147.0587,22.62,23,2,1.64,AU,1731385896
17,17,cabo san lucas,22.8909,-109.9124,21.20,66,29,5.89,MX,1731385652


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

hotel_df['Hotel Name'] = ''

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,atafu village,TK,-8.5421,-172.5159,72,
9,chaohu,CN,31.6000,117.8667,40,
13,geraldton,AU,-28.7667,114.6000,50,
14,new norfolk,AU,-42.7826,147.0587,23,
17,cabo san lucas,MX,22.8909,-109.9124,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
radius = 10000  # Radio de búsqueda en metros (10,000 metros)
params = {
    "categories": "accommodation.hotel",  
    "radius": radius,  
    "apiKey": geoapify_key  
}

print("Starting hotel search")

for index, row in hotel_df.iterrows():
  
    latitude = row["Lat"]
    longitude = row["Lng"]

    
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    base_url = "https://api.geoapify.com/v2/places"

   
    response = requests.get(base_url, params=params)

    name_address = response.json()

    try:
        hotel_name = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name 
    except (KeyError, IndexError):
       
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


hotel_df.head()

Starting hotel search
atafu village - nearest hotel: Luana Liki Hotel
chaohu - nearest hotel: 汉爵华阳酒店
geraldton - nearest hotel: Ocean Centre Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
cabo san lucas - nearest hotel: Comfort Rooms
blackmans bay - nearest hotel: Villa Howden
tura - nearest hotel: Hotel Polo Orchid
fale old settlement - nearest hotel: Luana Liki Hotel
enewetak - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
dakhla - nearest hotel: Hotel Ayouz
sittwe - nearest hotel: Yuzana Aung Motel 1
sarangani - nearest hotel: Juanita's Inn
si satchanalai - nearest hotel: Tonpalm Resort
segou - nearest hotel: No hotel found
douentza - nearest hotel: Hotel Camp Amitie
tazacorte - nearest hotel: App Leyma
manokwari - nearest hotel: Hotel Arfak
andergrove - nearest hotel: No hotel found
kalmunai - nearest hotel: Saji new food
nishinoomote - nearest hotel: ゼウスハウス
myitkyina - nearest hotel: New Light Hotel
launceston - nearest hotel: Hotel Charl

,City,Country,Lat,Lng,Humidity,Hotel Name
1,atafu village,TK,-8.5421,-172.5159,72,Luana Liki Hotel
9,chaohu,CN,31.6000,117.8667,40,汉爵华阳酒店
13,geraldton,AU,-28.7667,114.6000,50,Ocean Centre Hotel
14,new norfolk,AU,-42.7826,147.0587,23,Woodbridge on the Derwent
17,cabo san lucas,MX,22.8909,-109.9124,66,Comfort Rooms


In [30]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Ganpati Palace',
    'country': 'India',
    'country_code': 'in',
    'state': 'Odisha',
    'state_district': 'Nuapada District',
    'county': 'Jonk',
    'city': 'Jonk',
    'suburb': 'Mograpali',
    'street': 'NH353',
    'lon': 82.511311,
    'lat': 20.8917206,
    'state_code': 'OR',
    'formatted': 'Ganpati Palace, NH353, Mograpali, Jonk -, Odisha, India',
    'address_line1': 'Ganpati Palace',
    'address_line2': 'NH353, Mograpali, Jonk -, Odisha, India',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database License',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'Ganpati Palace',
      'osm_id': 5257362512,
      'tourism': 'hotel',
      'osm_type': 'n'}},
    'distance': 53727,
    'place_id': '51

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [36]:
%%capture --no-display

hotel_df['hover'] = hotel_df['City'] + ', ' + hotel_df['Country'] + '<br>Hotel: ' + hotel_df['Hotel Name']

map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',  #
    geo=True, 
    color='Humidity',  
    size=10,  
    hover_cols=['hover'],  
    title="City Locations with Hotels"
)

# Mostrar el mapa
map_plot

:Points   [Lng,Lat]   (Humidity,hover)